In [86]:
# Merged files 
import os
import pandas as pd
import numpy as np

In [87]:
dirpath = os.getcwd()
features_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir),"udder_processing",  "udder_features")
dfv1 = pd.read_csv(os.path.join(features_dir, "volumes.csv"), index_col=0)
dfr1 = pd.read_csv(os.path.join(features_dir, "region_props.csv"))
dfr1["lp_group"] = "l"
dfv1["lp_group"] = "l"
dfv2 = pd.read_csv(os.path.join(features_dir, "volumes_newcows.csv"), index_col=0)
dfr2 = pd.read_csv(os.path.join(features_dir, "region_props_newcows.csv"))
dfr2["lp_group"] = "p"
dfv2["lp_group"] = "p"

dfr = pd.concat([dfr1, dfr2], axis = 0, ignore_index = True)
dfv = pd.concat([dfv1, dfv2], axis = 0, ignore_index = True)

dfg =  pd.read_csv("geodesic_all.csv")

df_all = dfr.merge(dfv, on=["cow","filename", "lp_group"])
df_all = dfg.merge(df_all, on=["cow","filename", "lp_group"])


In [88]:
features = df_all.columns.drop(["cow", "lp_group", "filename"])

In [89]:
id_vars = ["cow", "lp_group"]
df_all2= pd.melt(df_all, id_vars = id_vars, value_vars = features)
grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean])
grouped.columns = [b.replace("nan", "") if b else a for a, b in grouped.columns]
grouped = grouped.reset_index()

C:\Users\marie\AppData\Local\Temp\ipykernel_12676\3526364812.py:3: FutureWarning: The provided callable <function nanmedian at 0x00000277C694F560> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean])
C:\Users\marie\AppData\Local\Temp\ipykernel_12676\3526364812.py:3: FutureWarning: The provided callable <function nanmin at 0x00000277C694E840> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean])
C:\Users\marie\AppData\Local\Temp\ipykernel_12676\35263

In [90]:
len(grouped)

4002

In [91]:

id_vars = ["cow", "lp_group"]
df_all2= pd.melt(df_all, id_vars = id_vars, value_vars = features).reset_index()
grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean, "count"])
grouped.columns = [b.replace("nan", "") if b else a for a, b in grouped.columns]
grouped = grouped.reset_index()

grouped_wide = grouped[["cow", "lp_group", "variable", "median"]].pivot(index = id_vars, columns = "variable", values = "median").reset_index()
grouped_wide2 = grouped[["cow", "lp_group", "variable", "mean"]].pivot(index = id_vars, columns = "variable", values = "mean").reset_index()

grouped_wide.to_csv("udder_features_median.csv", index = False)
grouped_wide2.to_csv("udder_features_mean.csv", index = False)
df_all.to_csv("udder_features_raw.csv", index = False)
grouped.to_csv("udder_features_grouped.csv", index = False)

C:\Users\marie\AppData\Local\Temp\ipykernel_12676\3202310750.py:3: FutureWarning: The provided callable <function nanmedian at 0x00000277C694F560> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean, "count"])
C:\Users\marie\AppData\Local\Temp\ipykernel_12676\3202310750.py:3: FutureWarning: The provided callable <function nanmin at 0x00000277C694E840> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  grouped = df_all2[["cow","lp_group" ,"variable", "value"]].groupby(["cow", "lp_group", "variable"]).agg([np.nanmedian, np.nanmin,np.nanmax,np.nanmean, "count"])
C:\Users\marie\AppData\Local\Temp\ipy

In [78]:
counts = df_all[["cow", "filename"]].groupby("cow").agg("count").reset_index()

In [79]:
np.mean(counts.filename)

170.54347826086956

In [80]:
np.std(counts.filename)

191.67942438568946

In [67]:
np.min(counts.filename)

1

In [68]:
np.max(counts.filename)

631